In [3]:
# Reference
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#4whatdoesldado
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
import pandas as pd

df = pd.read_csv("Sample 2001_2011/2001-2011-Table 1.csv")

In [4]:
df[['Company','Overview Text']]

,Company,Overview Text
0,Riverstone Networks,We are a leading provider of Internet infrastr...
1,Instinet Group LLC,We are the world's largest electronic agency s...
2,Alliance Data Systems,We are a leading provider of transaction servi...
3,Simplex Solutions,We provide software and services for integrate...
4,BAM! Entertainment,We are a rapidly emerging developer and publis...
...,...,...
117,Imperva,We provide home improvement information and se...
118,Angie's List,We operate a consumer-driven solution for our ...
119,Jive Software,Jive’s mission is to change the way that work ...
120,Boingo Wireless Inc,Boingo makes it simple to connect to the mobil...


In [5]:
import gensim
from gensim.models import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bohaocao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    """
    Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
    Words are stemmed — words are reduced to their root form.
    """
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    """
    Words that have fewer than 3 characters are removed.
    All stopwords are removed.
    """
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
processed_docs = df['Overview Text'].map(preprocess)

In [8]:
"""
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
Filter:
less than 15 documents (absolute number) or
more than 0.5 documents (fraction of total corpus size, not absolute number).
after the above two steps, keep only the first 100000 most frequent tokens.
"""
dictionary = gensim.corpora.Dictionary(processed_docs)

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [9]:
"""
For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.
"""
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [10]:
def create_model_and_report_metrics(corpus, is_tfidf, num_topics, texts, id2word):
    model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=id2word, passes=2, workers=4)

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"model artifacts: num_topics:{num_topics}, is_tfidf:{is_tfidf}")
    print(f'Coherence Score for: ', coherence_lda)

    # a measure of how good the model is. lower the better.
    print(f'Perplexity: {model.log_perplexity(corpus)}')  

    return model


In [12]:
num_topics_choices = [5,10,15,20,25,30]
for num_topics in num_topics_choices:
    create_model_and_report_metrics(bow_corpus, False, num_topics, processed_docs, dictionary)

model artifacts: num_topics:5, is_tfidf:False
Coherence Score for:  0.2720587551107571
Perplexity: -5.7503308697809254
model artifacts: num_topics:10, is_tfidf:False
Coherence Score for:  0.2978269505678698
Perplexity: -5.827677283437225
model artifacts: num_topics:15, is_tfidf:False
Coherence Score for:  0.2972552215639094
Perplexity: -5.942355759226626
model artifacts: num_topics:20, is_tfidf:False
Coherence Score for:  0.28326549617851404
Perplexity: -6.08299227299289
model artifacts: num_topics:25, is_tfidf:False
Coherence Score for:  0.2987650974396573
Perplexity: -6.197751448790713
model artifacts: num_topics:30, is_tfidf:False
Coherence Score for:  0.28256270775351816
Perplexity: -6.3431941273718655


In [12]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [15]:
num_topics_choices = [5,10,15,20,25, 30]
for num_topics in num_topics_choices:
    create_model_and_report_metrics(corpus_tfidf, True, num_topics, processed_docs, dictionary)

model artifacts: num_topics:5, is_tfidf:True
Coherence Score for:  0.22482466034977894
Perplexity: -7.621217156608712
model artifacts: num_topics:10, is_tfidf:True
Coherence Score for:  0.24591223680551427
Perplexity: -9.033099534341979
model artifacts: num_topics:15, is_tfidf:True
Coherence Score for:  0.24636828108698008
Perplexity: -10.669660227886535
model artifacts: num_topics:20, is_tfidf:True
Coherence Score for:  0.2208224812445641
Perplexity: -10.61681605558313
model artifacts: num_topics:25, is_tfidf:True
Coherence Score for:  0.2439456411198218
Perplexity: -12.040754125760753
model artifacts: num_topics:30, is_tfidf:True
Coherence Score for:  0.2583301841544974
Perplexity: -12.314428811655548


In [15]:
corpus_tfidf_8 = create_model_and_report_metrics(corpus_tfidf, True, 8, processed_docs, dictionary)

model artifacts: num_topics:8, is_tfidf:True
Coherence Score for:  0.24033243088083076
Perplexity: -8.291428334698


In [27]:
corpus_tfidf_15 = create_model_and_report_metrics(corpus_tfidf, True, 15, processed_docs, dictionary)

model artifacts: num_topics:15, is_tfidf:True
Coherence Score for:  0.22033667877168436
Perplexity: -10.50494640902069


In [17]:
corpus_tfidf_5 = create_model_and_report_metrics(corpus_tfidf, True, 5, processed_docs, dictionary)

model artifacts: num_topics:5, is_tfidf:True
Coherence Score for:  0.2525193179978911
Perplexity: -7.468032459587686


In [24]:
model_5 = create_model_and_report_metrics(bow_corpus, False, 5, processed_docs, dictionary)

model artifacts: num_topics:5, is_tfidf:False
Coherence Score for:  0.24266101376693414
Perplexity: -5.731501961453066


In [20]:
# Visualize the topics
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(corpus_tfidf_8, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.022498  0.017992       1        1  75.827451
6     -0.034226  0.022261       2        1   5.800263
0     -0.013207  0.006490       3        1   5.617512
7      0.002191 -0.030589       4        1   3.850535
2     -0.014918 -0.017984       5        1   2.904082
3      0.035153  0.006532       6        1   2.464247
5      0.000116 -0.010185       7        1   1.836666
1      0.002394  0.005483       8        1   1.699245, topic_info=          Term      Freq     Total Category  logprob  loglift
149     client  8.000000  8.000000  Default  30.0000  30.0000
223     member  5.000000  5.000000  Default  29.0000  29.0000
323      mobil  4.000000  4.000000  Default  28.0000  28.0000
335   advertis  7.000000  7.000000  Default  27.0000  27.0000
294  enterpris  5.000000  5.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
38      acquir  0.049748  2.784619   Topic8  -5.5752   0.0501
335   advertis  0.057356  7.280734   Topic8  -5.4329  -0.7687
9      content  0.052340  4.586091   Topic8  -5.5244  -0.3980
294  enterpris  0.052530  5.675429   Topic8  -5.5208  -0.6075
113   platform  0.050911  5.232566   Topic8  -5.5521  -0.5576

[412 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
310       1  0.707632      abl
165       1  0.929317   accord
38        1  0.718231   acquir
39        1  0.644996    activ
1         1  0.729727      add
...     ...       ...      ...
207       1  0.949011   vendor
272       1  0.948576  visitor
145       1  0.918780    volum
164       1  0.802931     work
146       1  0.663311    world

[192 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 1, 8, 3, 4, 6, 2])

In [30]:
vis = pyLDAvis.gensim.prepare(corpus_tfidf_5, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.004867  0.018656       1        1  81.886242
3     -0.026836 -0.008010       2        1   7.199386
1      0.012012 -0.018982       3        1   4.394519
4     -0.001647  0.013167       4        1   3.555274
2      0.011605 -0.004832       5        1   2.964580, topic_info=          Term      Freq     Total Category  logprob  loglift
294  enterpris  5.000000  5.000000  Default  30.0000  30.0000
335   advertis  3.000000  3.000000  Default  29.0000  29.0000
149     client  2.000000  2.000000  Default  28.0000  28.0000
34        user  4.000000  4.000000  Default  27.0000  27.0000
176     consum  4.000000  4.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
230    septemb  0.120516  3.827833   Topic5  -5.2469   0.0602
8      connect  0.115878  3.926791   Topic5  -5.2862  -0.0046
211      brand  0.111042  2.857182   Topic5  -5.3288   0.2708
118    process  0.113153  5.545738   Topic5  -5.3100  -0.3736
4        build  0.109323  2.972761   Topic5  -5.3444   0.2155

[262 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
165       1  0.936319    accord
0         1  0.738759   account
1         1  0.794323       add
2         1  0.576096    advanc
166       1  0.807201  advantag
...     ...       ...       ...
260       1  0.821161      view
271       1  0.923922     visit
272       1  0.912650   visitor
164       1  0.636393      work
146       1  0.721147     world

[162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 3])

In [26]:
vis = pyLDAvis.gensim.prepare(model_5, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.026544  0.035994       1        1  27.514077
0      0.034071  0.005199       2        1  25.341427
1     -0.032147 -0.024319       3        1  19.005537
3      0.024434 -0.000015       4        1  15.429106
2      0.000185 -0.016859       5        1  12.709854, topic_info=           Term        Freq       Total Category  logprob  loglift
280       email   99.000000   99.000000  Default  30.0000  30.0000
335    advertis  171.000000  171.000000  Default  29.0000  29.0000
326      fiscal   48.000000   48.000000  Default  28.0000  28.0000
297  profession  104.000000  104.000000  Default  27.0000  27.0000
20         host   51.000000   51.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
46      billion   15.528078  106.573122   Topic5  -5.0561   0.1366
14       demand   15.117070   96.968328   Topic5  -5.0829   0.2042
83       inform   16.471790  212.535220   Topic5  -4.9971  -0.4947
33       tradit   15.232786  107.959294   Topic5  -5.0753   0.1045
303     support   15.069916   95.688374   Topic5  -5.0860   0.2144

[288 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.279819  account
0         2  0.376308  account
0         3  0.164032  account
0         4  0.096489  account
0         5  0.086840  account
...     ...       ...      ...
146       1  0.077597    world
146       2  0.329787    world
146       3  0.077597    world
146       4  0.232791    world
146       5  0.271589    world

[950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])

In [31]:
model_10 = create_model_and_report_metrics(bow_corpus, False, 10, processed_docs, dictionary)
vis = pyLDAvis.gensim.prepare(model_10, bow_corpus, dictionary)
vis

model artifacts: num_topics:10, is_tfidf:False
Coherence Score for:  0.3301357369722567
Perplexity: -5.815200315092786


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.018555  0.041825       1        1  21.456798
1      0.048714 -0.009682       2        1  13.581083
5      0.006965 -0.031241       3        1  13.065931
2     -0.029007  0.013260       4        1  12.409561
4     -0.069614  0.001128       5        1  11.065740
9      0.016469  0.010340       6        1   7.534071
6      0.015249 -0.086879       7        1   7.061689
0      0.042009  0.012992       8        1   5.309207
3     -0.080716  0.004719       9        1   4.531786
8      0.031376  0.043536      10        1   3.984134, topic_info=             Term        Freq       Total Category  logprob  loglift
280         email   90.000000   90.000000  Default  30.0000  30.0000
244          home  122.000000  122.000000  Default  29.0000  29.0000
149        client  105.000000  105.000000  Default  28.0000  28.0000
335      advertis  171.000000  171.000000  Default  27.0000  27.0000
323         mobil  102.000000  102.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
292          data    6.632977  197.462051  Topic10  -4.7466  -0.1706
211         brand    5.448000   89.640086  Topic10  -4.9434   0.4223
159  relationship    5.480641   97.835596  Topic10  -4.9374   0.3408
243        growth    5.550392  109.235847  Topic10  -4.9248   0.2432
34           user    5.413531  182.707443  Topic10  -4.9498  -0.2961

[507 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
310       1  0.161697   abl
310       2  0.161697   abl
310       3  0.107798   abl
310       4  0.080848   abl
310       5  0.188646   abl
...     ...       ...   ...
164       5  0.028393  work
164       6  0.028393  work
164       7  0.141963  work
164       8  0.113570  work
164       9  0.113570  work

[2461 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 6, 3, 5, 10, 7, 1, 4, 9])

In [34]:
model_10.print_topics()

[(0,
  '0.030*"user" + 0.028*"advertis" + 0.028*"mobil" + 0.017*"connect" + 0.014*"devic" + 0.012*"consum" + 0.012*"media" + 0.012*"comput" + 0.011*"design" + 0.011*"platform"'),
 (1,
  '0.021*"content" + 0.019*"purchas" + 0.017*"user" + 0.017*"process" + 0.016*"inform" + 0.014*"mobil" + 0.012*"search" + 0.011*"consum" + 0.011*"advertis" + 0.011*"interact"'),
 (2,
  '0.026*"payment" + 0.025*"process" + 0.017*"account" + 0.014*"transact" + 0.012*"secur" + 0.012*"host" + 0.012*"system" + 0.012*"total" + 0.011*"design" + 0.010*"quarter"'),
 (3,
  '0.035*"client" + 0.026*"invest" + 0.022*"financi" + 0.021*"incom" + 0.021*"platform" + 0.019*"total" + 0.016*"fiscal" + 0.016*"secur" + 0.015*"loss" + 0.014*"account"'),
 (4,
  '0.029*"septemb" + 0.026*"retail" + 0.020*"capit" + 0.017*"gain" + 0.016*"repres" + 0.016*"decemb" + 0.015*"platform" + 0.013*"respect" + 0.013*"volum" + 0.012*"total"'),
 (5,
  '0.027*"project" + 0.025*"improv" + 0.023*"advertis" + 0.021*"inform" + 0.020*"home" + 0.013*"

In [50]:
lda_model = corpus_tfidf_5
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8113599419593811	 
Topic: 0.008*"process" + 0.008*"payment" + 0.007*"price" + 0.007*"enterpris" + 0.007*"order" + 0.006*"transact" + 0.006*"retail" + 0.006*"data" + 0.006*"design" + 0.006*"manufactur"

Score: 0.17300157248973846	 
Topic: 0.020*"client" + 0.010*"marketplac" + 0.009*"home" + 0.009*"member" + 0.008*"mobil" + 0.008*"digit" + 0.008*"quarter" + 0.008*"septemb" + 0.007*"email" + 0.007*"local"


In [61]:
s = """0.008*"process" + 0.008*"payment" + 0.007*"price" + 0.007*"enterpris" + 0.007*"order" + 0.006*"transact" + 0.006*"retail" + 0.006*"data" + 0.006*"design" + 0.006*"manufactur"""
[*s.split("+")][0].split("*")[1].strip()

'"process"'

In [117]:
# bow_corpus
# corpus_tfidf

def output_topics(model, is_tfidf):
    corpus = corpus_tfidf if is_tfidf else bow_corpus
    d = {"index": [], "score":[], "topic":[], "prob": []}
    for item_index, item in enumerate(corpus):
        for index, score in sorted(model[item], key=lambda tup: -1*tup[1]):
            topic_str = model.print_topic(index, 10)
            #print(topic_str)
            topic_scores_str = [*topic_str.split("+")]
            #print(topic_scores_str)
            for topic_score_str in topic_scores_str:
                s =[*topic_score_str.split("*")]
                #print(s)
                prob = float(s[0].strip())
                topic = s[1].strip()
                d['index'].append(item_index)
                d['score'].append(score)
                d['topic'].append(topic)
                d['prob'].append(prob)
            break
    return d

In [118]:
d = output_topics(corpus_tfidf_8, True)
corpus_tfidf_8_result = pd.DataFrame(d)
corpus_tfidf_8_result

,index,score,topic,prob
0,0,0.675611,"""client""",0.013
1,0,0.675611,"""advertis""",0.011
2,0,0.675611,"""member""",0.009
3,0,0.675611,"""communic""",0.009
4,0,0.675611,"""interact""",0.008
...,...,...,...,...
1215,121,0.871492,"""email""",0.008
1216,121,0.871492,"""channel""",0.008
1217,121,0.871492,"""digit""",0.008
1218,121,0.871492,"""profession""",0.008


In [119]:
d = output_topics(model_10, True)
model_10_result = pd.DataFrame(d)
model_10_result

,index,score,topic,prob
0,0,0.861899,"""client""",0.018
1,0,0.861899,"""channel""",0.017
2,0,0.861899,"""sell""",0.014
3,0,0.861899,"""tradit""",0.013
4,0,0.861899,"""interact""",0.013
...,...,...,...,...
1215,121,0.868080,"""perform""",0.015
1216,121,0.868080,"""platform""",0.013
1217,121,0.868080,"""design""",0.012
1218,121,0.868080,"""demand""",0.011


In [121]:
model_10_result[model_10_result['index']==1]

,index,score,topic,prob
10,1,0.901234,"""payment""",0.026
11,1,0.901234,"""process""",0.025
12,1,0.901234,"""account""",0.017
13,1,0.901234,"""transact""",0.014
14,1,0.901234,"""secur""",0.012
15,1,0.901234,"""host""",0.012
16,1,0.901234,"""system""",0.012
17,1,0.901234,"""total""",0.012
18,1,0.901234,"""design""",0.011
19,1,0.901234,"""quarter""",0.010
